C:\Users\strik\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe


In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Set device to GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA device:", device)
else:
    print("CUDA not available, using CPU")

ModuleNotFoundError: No module named 'torchvision'

In [2]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}
'''
label_encodings = {
    'nv': 0,
    'mel': 1,
    'bkl': 2,
    'bcc': 3,
    'akiec': 4,
    'vasc': 5,
    'df': 6
}
'''

"\nlabel_encodings = {\n    'nv': 0,\n    'mel': 1,\n    'bkl': 2,\n    'bcc': 3,\n    'akiec': 4,\n    'vasc': 5,\n    'df': 6\n}\n"

In [3]:
df = pd.read_csv('/work/HAM10000_metadata.csv')
df

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [4]:
df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age             57
sex              0
localization     0
dtype: int64

In [5]:
# fill null age values with mean age
df.fillna((df['age'].mean()), inplace=True)

In [6]:
label_encoder = LabelEncoder()
'''
df_input = df.drop(["dx", "lesion_id", "dx_type"], axis=1)
df_input["sex"] = label_encoder.fit_transform(df_input["sex"])
df_input["localization"] = label_encoder.fit_transform(df_input["localization"])

'''
df_output = df["dx"]
df_output = label_encoder.fit_transform(df_output)


In [7]:
# Params

# Data Loader
batch_size = 16
num_workers = 2
im_width, im_height = 224, 224
train_dir= '/work/skin_dataset'
validation_dir = '/work/skin_dataset' # TODO: Create directory for validation images

In [8]:
import csv

def csv_to_dict(csv_file_path):
    image_to_dx = {}
    with open(csv_file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for index,row in enumerate(csv_reader):
            image_to_dx[row['image_id']] = df_output[index] #label_encodings[row['dx']]
    
    return image_to_dx

In [9]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, labels, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.labels = labels
        self.image_names = [f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        im_name = self.image_names[idx]
        im_label = self.labels[im_name.rsplit('.', 1)[0]]
        
        im_path = os.path.join(self.root_dir, im_name)
        image = Image.open(im_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return {
            "image": image,
            "label": im_label,
        }

transform = transforms.Compose([
    transforms.Resize((im_width, im_height)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
dataset = ImageDataset(root_dir=train_dir, labels=csv_to_dict('/work/HAM10000_metadata.csv'), transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Initialize data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [14]:
from torch import nn, optim

input_size =  224
learning_rate = 0.001
amount_epochs = 20

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv = nn.Conv2d(3, 64, kernel_size=3)  # Example layer
        self.fc = nn.Linear(64, 7)  # Final linear layer

    def forward(self, x):
        x = self.conv(x)
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))  # Global average pooling
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

In [ ]:
model = NeuralNetwork()

loss = nn.CrossEntropyLoss() 
optimize = optim.Adam(model.parameters(), lr = learning_rate)

for epochs in range(amount_epochs):
    for i, batch in enumerate(train_loader):
        X, y = batch['image'], batch['label']
        result = model(X)
        lossResult = loss(result, y)
        optimize.zero_grad()
        lossResult.backward()
        optimize.step()
    
    print(f'Epoch [{epochs+1}/{amount_epochs}], Loss: {lossResult.item():.4f}')


Epoch [1/20], Loss: 0.6914
Epoch [2/20], Loss: 1.1265
Epoch [3/20], Loss: 1.5306
Epoch [4/20], Loss: 1.1065
Epoch [5/20], Loss: 1.1726
Epoch [6/20], Loss: 1.1047
Epoch [7/20], Loss: 1.2215
Epoch [8/20], Loss: 1.4272
Epoch [9/20], Loss: 1.4795
Epoch [10/20], Loss: 0.9359


In [12]:
model.eval()

total_size, acc = 0, 0

for i, batch in enumerate(validation_loader):
    X, y = batch['image'], batch['label']
    result = model(X)
    predicted_labels = torch.argmax(result, dim=1)
    total_size += predicted_labels.size(0)
    acc += sum(1 for predicted_val, actual_val in zip(predicted_labels, y) if predicted_val == actual_val)
print(acc/total_size)
    

0.7093023255813954


In [13]:
for batch in train_loader:
    print(batch.shape)

AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
for batch in train_loader:
    print(batch["image"].shape)

torch.Size([16, 3, 224, 224])
torch.Size([16, 3, 224, 224])


KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a5f9b5e9-c149-465b-a64a-460916aa0813' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>